In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os


In [2]:
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

def get_weight_variable(shape, regularizer):
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None: tf.add_to_collection('losses', regularizer(weights))
    return weights


def inference(input_tensor, regularizer):
    with tf.variable_scope('layer1'):

        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases

    return layer2

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH="MNIST_model/"
MODEL_NAME="mnist_model"


def train(mnist):

    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = inference(x, regularizer)
    global_step = tf.Variable(0, trainable=False)


    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)

    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))

    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')


    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            if i % 1000 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)


def main(argv=None):
    mnist = input_data.read_data_sets("datasets/MNIST_data", one_hot=True)
    train(mnist)

if __name__ == '__main__':
    tf.app.run()

W0815 15:34:12.889239 4322330048 deprecation.py:323] From <ipython-input-3-80f74580657f>:57: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0815 15:34:12.890468 4322330048 deprecation.py:323] From /Users/mac/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0815 15:34:12.891482 4322330048 deprecation.py:323] From /Users/mac/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instr

Extracting datasets/MNIST_data/train-images-idx3-ubyte.gz


W0815 15:34:13.108135 4322330048 deprecation.py:323] From /Users/mac/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0815 15:34:13.111104 4322330048 deprecation.py:323] From /Users/mac/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0815 15:34:13.155333 4322330048 deprecation.py:323] From /Users/mac/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and

Extracting datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting datasets/MNIST_data/t10k-labels-idx1-ubyte.gz


W0815 15:34:13.405082 4322330048 deprecation.py:323] From /Users/mac/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:433: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


After 1 training step(s), loss on training batch is 2.92458.
After 1001 training step(s), loss on training batch is 0.304043.
After 2001 training step(s), loss on training batch is 0.201294.
After 3001 training step(s), loss on training batch is 0.136044.
After 4001 training step(s), loss on training batch is 0.123777.


W0815 15:34:22.621626 4322330048 deprecation.py:323] From /Users/mac/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


After 5001 training step(s), loss on training batch is 0.103455.
After 6001 training step(s), loss on training batch is 0.106522.
After 7001 training step(s), loss on training batch is 0.0836013.
After 8001 training step(s), loss on training batch is 0.0813761.
After 9001 training step(s), loss on training batch is 0.0725381.
After 10001 training step(s), loss on training batch is 0.0648956.
After 11001 training step(s), loss on training batch is 0.0617109.
After 12001 training step(s), loss on training batch is 0.0729964.
After 13001 training step(s), loss on training batch is 0.0626243.
After 14001 training step(s), loss on training batch is 0.0521108.
After 15001 training step(s), loss on training batch is 0.0531973.
After 16001 training step(s), loss on training batch is 0.0547956.
After 17001 training step(s), loss on training batch is 0.0457016.
After 18001 training step(s), loss on training batch is 0.0464132.
After 19001 training step(s), loss on training batch is 0.0445985.
Af

SystemExit: 

/Users/mac/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:


BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARAZATION_RATE = 0.0001
TRAINING_STEPS = 40000
MOVING_AVERAGE_DECAY = 0.99


MODEL_SAVE_PATH = "MNIST_model/"
MODEL_NAME = "model.ckpt"

def train(mnist):
    x = tf.placeholder(
        tf.float32, [None,INPUT_NODE], name = 'x-input')
    
    y_true = tf.placeholder(
        tf.float32, [None,OUTPUT_NODE], name = 'y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARAZATION_RATE)

    y = inference( x , regularizer)

    global_step = tf.Variable(0, trainable = False)

    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)

    variables_averages_op = variable_averages.apply(
        tf.trainable_variables())

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = y, labels = tf.argmax(y_true, 1))

    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))

    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name = 'train')
    
        saver = tf.train.Saver()
        with tf.Session() as sess:
            print("b4")
            tf.global_variables_initializer().run()
            print("after")
        
            for i in range(TRAINING_STEPS):
                xs, ys = mnist.train.next_batch(BATCH_SIZE)
                print(xs.shape)
                _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict = {x:xs, y_true: ys})
            
                if i % 1000 == 0:
                    print ("After %d training steps, loss on training batch is %g" %(step, loss_value))
                
                    saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step = global_step)
                




In [10]:
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

mnist = input_data.read_data_sets("/Users/mac/Documents/DL_tenser_flow/mnist_data",one_hot = True)

train(mnist)


Extracting /Users/mac/Documents/DL_tenser_flow/mnist_data/train-images-idx3-ubyte.gz
Extracting /Users/mac/Documents/DL_tenser_flow/mnist_data/train-labels-idx1-ubyte.gz
Extracting /Users/mac/Documents/DL_tenser_flow/mnist_data/t10k-images-idx3-ubyte.gz
Extracting /Users/mac/Documents/DL_tenser_flow/mnist_data/t10k-labels-idx1-ubyte.gz


ValueError: Variable layer1/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-2-ea9c40841a29>", line 8, in get_weight_variable
    initializer = tf.truncated_normal_initializer(stddev = 0.1))
  File "<ipython-input-2-ea9c40841a29>", line 16, in inference
    [INPUT_NODE, LAYER1_NODE], regularizer)
  File "<ipython-input-3-4bb8cc76297a>", line 24, in train
    y = inference( x , regularizer)
  File "<ipython-input-4-1e57117562be>", line 6, in <module>
    train(mnist)
  File "/Users/mac/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
os.getcwd()